# TAO PTM Inference using TAO-Deploy

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080"> 

## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Download a TAO special use-case pre-trained model
* Export the downloaded etlt model to TensorRT engine
* Run inference on TRT engine file using TAO-Deploy
* Visualize the inferences

### Table of Contents

This notebook shows an example of classifying gestures using GestureNet in the Train Adapt Optimize (TAO) Toolkit.

1. [Get the TensorRT tar file](#head-1) <br>
2. [Setup GPU environment](#head-2) <br>
    2.1 [Connect to GPU Instance](#head-2-1) <br>
    2.2 [Mounting Google drive](#head-2-2) <br>
3. [Choose PTM model](#head-3) <br>
4. [Untar TensorRT file if needed](#head-4) <br>
5. [Install dependencies](#head-5) <br>
6. [Download the etlt model of the ptm chosen](#head-6) <br>
7. [Generate TRT engine file for the ptm's etlt file](#head-7) <br>
8. [Run inference on TRT engine file](#head-8) <br>
9. [Visualize inference](#head-8) <br>

#### FIXME
1. ptm_model_name - set this to one of the available PTM models
1. trt_tar_path - set this path of the uploaded TensorRT tar.gz file after browser download
1. trt_untar_folder_path - set to path of the folder where the TensoRT tar.gz file has to be untarred into
1. trt_version - set this to the version of TRT you have downloaded
1. COLAB_NOTEBOOKS_PATH - set this to the path of cloned colab notebook's github folder
1. GENERAL_WHL_PATH: set this to the path contain eff whl files
1. CODEBASE_WHL_PATH: set this to the path contining tao_deploy whl files
1. ptm_download_folder: set the folder path where you want to download the file into
1. **(Optional FIXME)** model_to_download_map(dictionary_value for chosen PTM key) - change the version tag of the PTM model if you want a version different from the below defaults
1. data_type - choose between FP32 and FP16
1. trt_out_folder - set this to the output folder for TensorRT engine file writing
1. inference_out_folder - set this to the output folder to write the inference results to
1. inference_input_images_folder - set this to the folder path containing images to run the inference on

## 1. Get the TensorRT tar file

1. Visit https://developer.nvidia.com/tensorrt
2. Clicking `Download now` from step one directs you to https://developer.nvidia.com/nvidia-tensorrt-download where you have to Login/Join Now for Nvidia Developer Program Membership
3. Now, in the download page: Choose TensorRT 8 in available versions
4. Agree to Terms and Conditions
5. Click on TensorRT 8.5 GA to expand the available options
6. Click on 'TensorRT 8.5 GA for Linux x86_64 and CUDA 11.0, 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7 and 11.8 TAR Package' to dowload the TAR file
7. Upload the the tar file to your Google Drive

## 2. Setup GPU environment <a class="anchor" id="head-2"></a>

### 2.1 Connect to GPU Instance <a class="anchor" id="head-2-1"></a>

1. Move any data saved to the Colab Instance storage to Google Drive  
2. Change Runtime type to GPU by Runtime(Top Left tab)->Change Runtime Type->GPU(Hardware Accelerator)
3.   Then click on Connect (Top Right)



### 2.2 Mounting Google drive <a class="anchor" id="head-2-2"></a>
Mount your Google drive storage to this Colab instance

In [ ]:
try:
    import google.colab
    %env GOOGLE_COLAB=1
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    %env GOOGLE_COLAB=0
    print("Warning: Not a Colab Environment")

## 3. Choose PTM model <a class="anchor" id="head-3"></a>

In [ ]:
# Define model_name
# Available models (#FIXME 1):
# 1. PeopleNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:peoplenet
# 2. PeopleSemSegNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:peoplesemsegnet
# 3. TrafficCamNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:trafficcamnet
# 4. DashCamNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:dashcamnet
# 5. FaceDetectIR - https://ngc.nvidia.com/catalog/models/nvidia:tao:facedetectir
# 6. FaceDetect - https://ngc.nvidia.com/catalog/models/nvidia:tao:facedetect
# 7. VehicleMakeNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:vehiclemakenet
# 8. VehicleTypeNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:vehicletypenet
# 9. LicensePlateDetection - https://ngc.nvidia.com/catalog/models/nvidia:tao:lpdnet
# 10. LicensePlateRecognition - https://ngc.nvidia.com/catalog/models/nvidia:tao:lprnet

ptm_model_name = "PeopleNet" # FIXME1 (Add the model name from the above mentioned list)

## 4. Untar TensorRT tar file if needed <a class="anchor" id="head-4"></a>

1. Set the variable `trt_tarfile_path` to the path where you uploaded the TensorRT tar file
1. Set the folder you want to untar into in `trt_untar_folder_path`
1. Set the TRT version you have downloaded in `trt_version`
  - If the TRT tar file's name is TensorRT-8.5.1.7.Linux.x86_64-gnu.cuda-11.8.cudnn8.6.tar.gz, then the version is 8.5.1.7

In [ ]:
# FIXME 2: set this path of the uploaded TensorRT tar.gz file after browser download
trt_tar_path="/content/drive/MyDrive/TensorRT-8.5.1.7.Linux.x86_64-gnu.cuda-11.8.cudnn8.6.tar.gz"

import os
if not os.path.exists(trt_tar_path):
  raise Exception("TAR file not found in the provided path")

# FIXME 3: set to path of the folder where the TensoRT tar.gz file has to be untarred into
%env trt_untar_folder_path=/content/drive/MyDrive/trt_untar
# FIXME 4: set this to the version of TRT you have downloaded
%env trt_version=8.5.1.7

!mkdir -p $trt_untar_folder_path

import os

untar = True
for fname in os.listdir(os.environ.get("trt_untar_folder_path", None)):
  if fname.startswith("TensorRT-"+os.environ.get("trt_version")) and not fname.endswith(".tar.gz"):
    untar = False

if untar:
  !tar -xzf $trt_tar_path -C $trt_untar_folder_path

In [ ]:
if not os.path.exists(f'{os.environ.get("trt_untar_folder_path")}/TensorRT-{os.environ.get("trt_version")}'):
  raise Exception("TensorRT not untarred properly. Please download and untar properly")

## 5. Install dependencies <a class="anchor" id="head-5"></a>

In [ ]:
#FIXME 5 - COLAB_NOTEBOOKS_PATH: set this to the path of cloned colab notebook's github folder
%env COLAB_NOTEBOOKS_PATH=/content/drive/MyDrive/ColabNotebooks/

if not os.path.exists(os.environ["COLAB_NOTEBOOKS_PATH"]):
  raise Exception("Error, enter Colab notebooks repo path correctly")

#FIXME 6 - GENERAL_WHL_PATH: set this to the path contain eff whl files
%env GENERAL_WHL_PATH=/content/drive/MyDrive/tao_deploy/general_whl
#FIXME 7 - CODEBASE_WHL_PATH: set this to the path contining tao_deploy whl files
%env CODEBASE_WHL_PATH=/content/drive/MyDrive/tao_deploy/codebase_whl

if os.path.exists(os.environ["GENERAL_WHL_PATH"]) and os.path.exists(os.environ["CODEBASE_WHL_PATH"]):

  !sed -i "s|PATH_TO_TRT|$trt_untar_folder_path|g" $COLAB_NOTEBOOKS_PATH/ptm/setup_env_colab.sh
  !sed -i "s|TRT_VERSION|$trt_version|g" $COLAB_NOTEBOOKS_PATH/ptm/setup_env_colab.sh
  !sed -i "s|PATH_TO_CODEBASE_WHL|$CODEBASE_WHL_PATH|g" $COLAB_NOTEBOOKS_PATH/ptm/setup_env_colab.sh
  !sed -i "s|PATH_TO_GENERAL_WHL|$GENERAL_WHL_PATH|g" $COLAB_NOTEBOOKS_PATH/ptm/setup_env_colab.sh

  !sh $COLAB_NOTEBOOKS_PATH/ptm/setup_env_colab.sh

else:
    raise Exception("Error, enter the whl paths correctly")

In [ ]:
import os
if os.environ.get("LD_LIBRARY_PATH","") == "":
  os.environ["LD_LIBRARY_PATH"] = ""
trt_lib_path = f':{os.environ.get("trt_untar_folder_path")}/TensorRT-{os.environ.get("trt_version")}/lib'
os.environ["LD_LIBRARY_PATH"]+=trt_lib_path

## 6. Download the etlt model of the ptm chosen <a class="anchor" id="head-6"></a>

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env LOCAL_PROJECT_DIR=/ngc_content/
%env CLI=ngccli_cat_linux.zip
!sudo mkdir -p $LOCAL_PROJECT_DIR/ngccli && chmod -R 777 $LOCAL_PROJECT_DIR

# Remove any previously existing CLI installations
!sudo rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u -q "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))
!cp /usr/lib/x86_64-linux-gnu/libstdc++.so.6 $LOCAL_PROJECT_DIR/ngccli/ngc-cli/libstdc++.so.6

In [ ]:
#List the available versions of the PTM model chosen

modelname_to_available_ptm_map = {
  "PeopleNet" : "nvidia/tao/peoplenet:pruned_v*",
  "PeopleSegNet" : "nvidia/tao/peoplesegnet:deployable*",
  "PeopleSemSegNet" : "nvidia/tao/peoplesemsegnet:deployable*",
  "TrafficCamNet" : "nvidia/tao/trafficcamnet:pruned_v*",
  "DashCamNet" : "nvidia/tao/dashcamnet:pruned_v*",
  "VehicleMakeNet" : "nvidia/tao/vehiclemakenet:pruned_v*",
  "VehicleTypeNet" : "nvidia/tao/vehicletypenet:pruned_v*",
  "LicensePlateRecognition" : "nvidia/tao/lprnet:deployable*",
  "LicensePlateDetection" : "nvidia/tao/lpdnet:pruned_v*",
  "FaceDetect" : "nvidia/tao/facenet:pruned_v*",
  "FaceDetectIR" : "nvidia/tao/facedetectir:pruned_v*"
}

model_to_view_regex = modelname_to_available_ptm_map[ptm_model_name]

# You can see different version of this PTM model
!ngc registry model list $model_to_view_regex

In [ ]:
# FIXME 8 - ptm_download_folder: set the folder path where you want to download the file into
%env ptm_download_folder=/content/drive/MyDrive/ptm_models/

!sudo rm -rf $ptm_download_folder
!sudo mkdir -p $ptm_download_folder && chmod -R 777 $ptm_download_folder

# Optional FIXME 9
# model_to_download_map dictionary_value for chosen PTM key - change the version tag of the PTM model if you want a version different from the below defaults
# Use only the versions which has a etlt file:
    # Go to the link of the PTM you chose
    # Click 'File Browser tab' to fiew the files for each version
    # Choose a version you want to override which contains a etlt file
# 1. PeopleNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:peoplenet
# 2. PeopleSemSegNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:peoplesemsegnet
# 3. TrafficCamNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:trafficcamnet
# 4. DashCamNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:dashcamnet
# 5. FaceDetectIR - https://ngc.nvidia.com/catalog/models/nvidia:tao:facedetectir
# 6. FaceDetect - https://ngc.nvidia.com/catalog/models/nvidia:tao:facedetect
# 7. VehicleMakeNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:vehiclemakenet
# 8. VehicleTypeNet - https://ngc.nvidia.com/catalog/models/nvidia:tao:vehicletypenet
# 9. LicensePlateDetection - https://ngc.nvidia.com/catalog/models/nvidia:tao:lpdnet
# 10. LicensePlateRecognition - https://ngc.nvidia.com/catalog/models/nvidia:tao:lprnet


model_to_download_map = {
  "PeopleNet" : "nvidia/tao/peoplenet:pruned_v2.3",
  "PeopleSemSegNet" : "nvidia/tao/peoplesemsegnet:deployable_vanilla_unet_v2.0.1",
  "TrafficCamNet" : "nvidia/tao/trafficcamnet:pruned_v1.0.2",
  "DashCamNet" : "nvidia/tao/dashcamnet:pruned_v1.0.2",
  "VehicleMakeNet" : "nvidia/tao/vehiclemakenet:pruned_v1.0.1",
  "VehicleTypeNet" : "nvidia/tao/vehicletypenet:pruned_v1.0.1",
  "LicensePlateRecognition" : "nvidia/tao/lprnet:deployable_v1.0",
  "LicensePlateDetection" : "nvidia/tao/lpdnet:pruned_v1.0",
  "FaceDetect" : "nvidia/tao/facenet:pruned_v2.0",
  "FaceDetectIR" : "nvidia/tao/facedetectir:pruned_v1.0.1"
}
model_to_download = model_to_download_map[ptm_model_name]
os.environ["model_to_download"] = model_to_download

!ngc registry model download-version $model_to_download --dest $ptm_download_folder

## 7. Generate TRT engine file for the ptm's etlt file <a class="anchor" id="head-7"></a>

In [ ]:
#FIXME 10 - data_type: choose FP32 or FP16
os.environ["data_type"] = "fp32"

#FIXME 11 - trt_out_folder: choose output folder for TensorRT engine file writing
trt_out_folder = "/content/drive/MyDrive/" + ptm_model_name

!mkdir -p $trt_out_folder

import glob
input_etlt_file_list = glob.glob(os.environ.get("ptm_download_folder")+"/**/*.etlt", recursive=True)
if len(input_etlt_file_list) == 0:
  raise Exception("ETLT file was not downloaded")

os.environ["input_etlt_file"] = input_etlt_file_list[0]

if ptm_model_name in ("LicensePlateRecognition","LicensePlateDetection"):
  # FIXME: country
  # us/ccpd for LicensePlateDetection - us for United States, ch for China
  # us/ch for LicensePlateRecongition - us for United States, ch for China

  country = "us"
  for countrywise_ptm in input_etlt_file_list:
    fname = countrywise_ptm.split("/")[-1]
    if fname.startswith(country):
      os.environ["input_etlt_file"] = countrywise_ptm

action = ""
if ptm_model_name in ("PeopleNet","LicensePlateDetection","DashCamNet","TrafficCamNet","FaceDetect","FaceDetectIR"):
  action = "_trt"

os.environ["KEY"] = "tlt_encode"

if ptm_model_name in ("LicensePlateRecognition","LicensePlateDetection"):
  os.environ["KEY"] = "nvidia_tlt"


os.environ["trt_experiment_spec"] = f"{os.environ.get('COLAB_NOTEBOOKS_PATH')}/ptm/specs/{ptm_model_name}/{ptm_model_name}{action}.txt"
os.environ["trt_out_file_name"] = f'{trt_out_folder}/{ptm_model_name}.trt.{os.environ["data_type"]}'

if ptm_model_name in ("PeopleNet","LicensePlateDetection","DashCamNet","TrafficCamNet","FaceDetect","FaceDetectIR"):
  !detectnet_v2 gen_trt_engine \
                    -m $input_etlt_file \
                    -k $KEY  \
                    -e $trt_experiment_spec \
                    --data_type $data_type \
                    --batch_size 1 \
                    --max_batch_size 1 \
                    --engine_file $trt_out_file_name
                
elif ptm_model_name in ("VehicleMakeNet","VehicleTypeNet"):
  !classification_tf1 gen_trt_engine \
                    -m $input_etlt_file \
                    -k $KEY  \
                    -e $trt_experiment_spec \
                    --data_type $data_type \
                    --batch_size 1 \
                    --max_batch_size 1 \
                    --batches 10 \
                    --engine_file $trt_out_file_name

elif ptm_model_name == "PeopleSemSegNet":
  !unet gen_trt_engine \
                    -m $input_etlt_file \
                    -k $KEY  \
                    -e $trt_experiment_spec \
                    --data_type $data_type \
                    --batch_size 1 \
                    --max_batch_size 3 \
                    --engine_file $trt_out_file_name

elif ptm_model_name == "LicensePlateRecognition":
  !lprnet gen_trt_engine \
                    -m $input_etlt_file \
                    -k $KEY  \
                    --data_type $data_type \
                    --min_batch_size 1 \
                    --opt_batch_size 4 \
                    --max_batch_size 16 \
                    --engine_file $trt_out_file_name

## 8. Run inference on TRT engine file <a class="anchor" id="head-8"></a>

In [ ]:
if ptm_model_name in ("PeopleNet","LicensePlateDetection","DashCamNet","TrafficCamNet","FaceDetect","FaceDetectIR"):
  action = "_infer"

os.environ["inference_experiment_spec"] = f"{os.environ.get('COLAB_NOTEBOOKS_PATH')}/ptm/specs/{ptm_model_name}/{ptm_model_name}{action}.txt"

# FIXME 12 - inference_out_folder: Folder path to write the inference results to
os.environ["inference_out_folder"] = "/content/drive/MyDrive/tao_ptm_inference/"
!rm -rf $inference_out_folder

# FIXME 13 - inference_input_images_folder: Folder path containing images to run the inference on
os.environ["inference_input_images_folder"] = f"/content/drive/MyDrive/tao_deploy_input_images/{ptm_model_name}"

print(f"Running inference for {ptm_model_name} on {os.environ['trt_out_file_name']}")
if ptm_model_name in ("PeopleNet","LicensePlateDetection","DashCamNet","TrafficCamNet","FaceDetect","FaceDetectIR"):
  !detectnet_v2 inference -e $inference_experiment_spec \
                                   -m $trt_out_file_name \
                                   -r $inference_out_folder \
                                   -i $inference_input_images_folder

elif ptm_model_name in ("VehicleMakeNet","VehicleTypeNet"):
  os.environ["inference_classmap"] = f"{os.environ.get('COLAB_NOTEBOOKS_PATH')}/ptm/specs/{ptm_model_name}/classmap.json"
  !classification_tf1 inference -e $inference_experiment_spec \
                                -m $trt_out_file_name \
                                -r $inference_out_folder \
                                -c $inference_classmap \
                                -i $inference_input_images_folder

elif ptm_model_name == "PeopleSemSegNet":
  # Write path of images to a file - it is required for PeopleSemSegNet which is based on unet 
  with open("/content/PeopleSemSegNet_inference.txt","w") as file_ptr:
    for image_name in os.listdir(os.environ["inference_input_images_folder"]):
      if image_name.endswith(".jpg") or image_name.endswith(".png"):
        file_ptr.write(os.environ["inference_input_images_folder"]+"/"+image_name+"\n")
        file_ptr.flush()

  !unet inference -e $inference_experiment_spec \
                  -m $trt_out_file_name \
                  -r $inference_out_folder
      
elif ptm_model_name == "LicensePlateRecognition":
  character_file_link = "https://api.ngc.nvidia.com/v2/models/nvidia/tao/lprnet/versions/trainable_v1.0/files/{}_lp_characters.txt".format(country)
  !wget -q -O /content/characters.txt $character_file_link

  !lprnet inference -i $inference_input_images_folder \
                      -e $inference_experiment_spec \
                      -m $trt_out_file_name

## 9. Visualize inference <a class="anchor" id="head-9"></a>

In [ ]:
if ptm_model_name in ("VehicleMakeNet", "VehicleTypeNet"):
  import pandas as pd
  dataframe = pd.read_csv(os.environ["inference_out_folder"]+ "/result.csv")
  with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    print(dataframe)

elif ptm_model_name != "LicensePlateRecognition":
  from IPython.display import Image, display
  import glob

  subfolder = ""
  if ptm_model_name == "PeopleSemSegNet":
    subfolder = "vis_overlay"

  inference_out_images_png = glob.glob(f'{os.environ["inference_out_folder"]}/{subfolder}/**/*.png', recursive=True)
  inference_out_images_jpg = glob.glob(f'{os.environ["inference_out_folder"]}/{subfolder}/**/*.jpg', recursive=True)
  inference_out_images = inference_out_images_png + inference_out_images_jpg

  if len(inference_out_images) == 0:
    raise Exception("Run Inference before visualization")

  display(Image(inference_out_images[0]))
